# Load Packages

In [1]:
using CSV
using DataFrames
using Gurobi
using Random
using Distances
using JuMP
using Gurobi

const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu


# Load the data

In [ ]:
model = Model(Gurobi.Optimizer)

# Load data
full_pairs = CSV.read("./data/kidney_data_with_states.csv", DataFrame)
centers = CSV.read("./data/Transplant_Center_Dataset.csv", DataFrame)

# # Take random samples
pair_size = 100
# center_size = 6
Random.seed!(42)
pairs = full_pairs[1:100, :]
# centers = centers[shuffle(1:nrow(centers))[1:min(center_size, nrow(centers))], :]

n = nrow(pairs)
m = nrow(centers)

println("Number of pairs: ", n)
println("Number of centers: ", m)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Number of pairs: 100
Number of centers: 256


# Base Model

In [ ]:
# Create Compatibility Matrix Based on Exact Blood Type Matches
c = zeros(Int, n, n)  # Initialize n x n matrix with zeros

# Create compatibility matrix (1 if compatible, 0 otherwise)
for i in 1:n
    donor = pairs.Donor[i]
    for j in 1:n
        patient = pairs.Patient[j]
        # Exact blood type match
        c[i, j] = donor == patient ? 1 : 0
    end
end

# Create JuMP Model with Gurobi Optimizer
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1) 

# Define Decision Variables: y[i,j] = 1 if donor i donates to patient j
@variable(model, y[1:n, 1:n], Bin)

# Define Objective: Maximize the Total Number of Transplants
@objective(model, Max, sum(y[i, j] for i in 1:n for j in 1:n))

# Add Compatibility Constraints: y[i,j] <= c[i,j]
@constraint(model, [i=1:n, j=1:n], y[i, j] <= c[i, j])

# Each Donor Can Donate to At Most One Patient: Σ y[i,j] <= 1, ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) <= 1)

# Each Patient Can Receive at Most One Kidney: Σ y[i,j] <= 1, ∀ j
@constraint(model, [j=1:n], sum(y[i, j] for i in 1:n) <= 1)

# Reciprocal Donation Constraints: Σ y[i,j] == Σ y[j,i], ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) == sum(y[j, i] for j in 1:n))

# Optimize the Model
optimize!(model)

# Check Optimization Status and Extract Results
status = termination_status(model)
if status == MOI.OPTIMAL
    println("\nOptimal solution found:\n")
    transplants = DataFrame(Donor = Int[], Patient = Int[])
    for i in 1:n
        for j in 1:n
            if value(y[i, j]) > 0.5
                println("Donor $(i) donates to Patient $(j)")
                push!(transplants, (Donor = i, Patient = j))
            end
        end
    end
    println("\nTotal optimal transplants: ", nrow(transplants))
    # display(transplants)
else
    println("\nNo optimal solution found. Status: ", status)
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 10300 rows, 10000 columns and 49800 nonzeros
Model fingerprint: 0x06bb70a9
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 10026 rows and 7726 columns
Presolve time: 0.03s
Presolved: 274 rows, 2274 columns, 8409 nonzeros
Varia

# Base Model with Altrustic Donors

# Introducing Matching Score

In [ ]:
# Compatibility matrix
comp = Dict()
blood_type_matches = [
    ("O", "O"), ("O", "A"), ("O", "B"), ("O", "AB"),
    ("A", "A"), ("A", "AB"),
    ("B", "B"), ("B", "AB"),
    ("AB", "AB")
]

for i in 1:n, j in 1:n
    donor = pairs[i, :Donor]
    patient = pairs[j, :Patient]
    comp[(i, j)] = (donor, patient) in blood_type_matches ? 1 : 0
end

# Introducing Transportation Cost

In [ ]:
# Create model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[1:n, 1:n, 1:m], Bin)  # x[i,j,k] = 1 if donor i donates to patient j through center k
@variable(model, y[1:m], Bin)  # y[k] = 1 if center k is used

# Calculate distances
distances = Dict()
for i in 1:n, k in 1:m
    pair_coords = (pairs[i, :lat], pairs[i, :lng])
    center_coords = (centers[k, :lat], centers[k, :lng])
    distances[(i, k)] = haversine(pair_coords, center_coords, 6371)  # 6371 is Earth's radius in km
end

# Hyper Parameter
# Calculate d_max from the sampled data
d_max = 0.0
for i in 1:n, j in 1:n, k in 1:m
    # Calculate total distance for this potential pairing through center k
    total_distance = distances[(i,k)] + distances[(j,k)]
    global d_max = max(d_max, total_distance)
end

# Calculate alpha based on d_max and number of pairs
α = 0

println("Maximum distance in sampled data (d_max): ", round(d_max, digits=2), " km")
println("Calculated α: ", α)


# Introducing Surgery Constrains